# Trabalho Prático 2

## Grupo 04 - Renato Garcia (A101987) & Bernardo Moniz (A102497)

## Problema 1
### Enunciado

1. Considere a descrição da cifra A5/1 que consta no documento +Lógica Computacional: a Cifra A5/1. Informação complementar pode ser obtida no artigo da Wikipedia.

   Pretende-se

   a. Definir e codificar, em Z3 e usando o tipo BitVec para modelar a informação, uma FSM que descreva o gerador de chaves.

   b. Considere as seguintes eventuais propriedades de erro:

      i. ocorrência de um “burst” $\,\mathsf{0}^t\,$  ($t$ zeros) que ocorre em $\,2^t\,$ passos ou menos.

      ii. ocorrência de um “burst” de tamanho $\,t\,$ que repete um “burst” anterior no mesmo output em $2^{t/2}$ passos ou menos.

   Tente codificar estas propriedades e verificar se são acessíveis a partir de um estado inicial aleatoriamente gerado.


### Resolução



KEY GENERATOR CIFRA A5/1

Neste trabalho, foram seguidas as indicações presentes nos documentos apresentados pelos docentes, bem como alguma informação online.

São feitas 64 + 22 + 100 iterações antes de ser gerada a keystream, tendo esta um valor fixo de 228 bits (este poderia ser alterado oferendo um valor k para ser gerada uma keystream com k bits).

Por fim, verifica-se as propriedades de erro solicitadas.

In [1]:
from z3 import * 
import random

def declare(step):
    """Declara as variáveis de estado para um dado passo"""
    return (
        BitVec(f'R1_{step}', 19),
        BitVec(f'R2_{step}', 22),
        BitVec(f'R3_{step}', 23)
    )

def init():
    """Inicializa os registadores R1, R2, e R3 com valores aleatórios"""
    R1_0, R2_0, R3_0 = declare(0)
    
    # Valores iniciais aleatórios
    r1_init = random.getrandbits(19)
    r2_init = random.getrandbits(22)
    r3_init = random.getrandbits(23)
    
    solver = Solver()
    solver.add(R1_0 == r1_init, R2_0 == r2_init, R3_0 == r3_init)
    
    return solver, R1_0, R2_0, R3_0

def trans(solver, step, R1_curr, R2_curr, R3_curr, key_bit=None, fc_bit=None, majority_rule=False, generate_keystream=False):
    """Define as transições de estado para uma etapa"""
    R1_next, R2_next, R3_next = declare(step + 1)

    fb1 = Extract(0, 0, R1_curr) ^ Extract(1, 1, R1_curr) ^ Extract(2, 2, R1_curr) ^ Extract(5, 5, R1_curr)
    fb2 = Extract(0, 0, R2_curr) ^ Extract(1, 1, R2_curr)
    fb3 = Extract(0, 0, R3_curr) ^ Extract(1, 1, R3_curr) ^ Extract(2, 2, R3_curr) ^ Extract(15, 15, R3_curr)
    
    if key_bit is not None:
        solver.add(
            R1_next == Concat(fb1 ^ key_bit, Extract(18, 1, R1_curr)),
            R2_next == Concat(fb2 ^ key_bit, Extract(21, 1, R2_curr)),
            R3_next == Concat(fb3 ^ key_bit, Extract(22, 1, R3_curr))
        )
    elif fc_bit is not None:
        solver.add(
            R1_next == Concat(fb1 ^ fc_bit, Extract(18, 1, R1_curr)),
            R2_next == Concat(fb2 ^ fc_bit, Extract(21, 1, R2_curr)),
            R3_next == Concat(fb3 ^ fc_bit, Extract(22, 1, R3_curr))
        )
    elif majority_rule or generate_keystream:
        C1, C2, C3 = Extract(10, 10, R1_curr), Extract(11, 11, R2_curr), Extract(12, 12, R3_curr)
        count_ones = If(C1 == 1, 1, 0) + If(C2 == 1, 1, 0) + If(C3 == 1, 1, 0)
        maj = If(count_ones >= 2, BitVecVal(1, 1), BitVecVal(0, 1))

        if generate_keystream:
            # Bits de saída para a keystream
            output_bit = Extract(0, 0, R1_curr) ^ Extract(0, 0, R2_curr) ^ Extract(0, 0, R3_curr)
            keystream_bit = BitVec(f'keystream_{step}', 1)
            solver.add(keystream_bit == output_bit)

        solver.add(
            If(C1 == maj, R1_next == Concat(fb1, Extract(18, 1, R1_curr)), R1_next == R1_curr),
            If(C2 == maj, R2_next == Concat(fb2, Extract(21, 1, R2_curr)), R2_next == R2_curr),
            If(C3 == maj, R3_next == Concat(fb3, Extract(22, 1, R3_curr)), R3_next == R3_curr)
        )
    
    return R1_next, R2_next, R3_next

def get_keystream_bytes(keystream_bits):
    """Converte a string de bits em bytes"""
    keystream_clean = keystream_bits.replace(" ", "")
    bytes_list = []
    for i in range(0, len(keystream_clean), 8):
        byte = keystream_clean[i:i+8]
        if len(byte) == 8:
            bytes_list.append(int(byte, 2))
    return bytes_list

def print_state_with_keystream(model, step, bits_r1, bits_r2, bits_r3, keystream_bits="", print_keystream=False):
    """Função auxiliar para imprimir o estado dos registadores e keystream"""
    R1 = model[BitVec(f'R1_{step}', bits_r1)]
    R2 = model[BitVec(f'R2_{step}', bits_r2)]
    R3 = model[BitVec(f'R3_{step}', bits_r3)]
    
    print(f"\nEstado {step}:")
    print(f"R1: {format(R1.as_long(), f'0{bits_r1}b')}")
    print(f"R2: {format(R2.as_long(), f'0{bits_r2}b')}")
    print(f"R3: {format(R3.as_long(), f'0{bits_r3}b')}")
    
    if print_keystream:
        current_bit = model[BitVec(f'keystream_{step-1}', 1)].as_long()
        new_keystream = keystream_bits + str(current_bit)
        
        # Formatação da keystream em grupos de 8 bits
        formatted_keystream = ""
        for i in range(0, len(new_keystream), 8):
            formatted_keystream += new_keystream[i:i+8] + " "
            
        print("\nKeystream gerada até agora:")
        print(f"Em bits: {formatted_keystream.strip()}")
        
        # Mostra os bytes já completos
        bytes_list = get_keystream_bytes(new_keystream)
        if bytes_list:
            print(f"Em bytes (hex): {bytes(bytes_list).hex()}")
            print(f"Número de bytes gerados: {len(bytes_list)}")
        
        return new_keystream
    
    return keystream_bits
    
def verificar_burst_keystream(model, t, keystream_bits):
    """Verifica a ocorrência de um burst de t zeros na keystream até 2^t passos"""
    burst_size = 2 ** t
    burst_pattern = BitVecVal(0, t)  # Padrão de burst de t zeros
    max_steps = burst_size  # Limite máximo de passos a verificar

    solver_burst = Solver()

    print("\n=== Verificação de Burst na Keystream ===")
    print(f"Verificando bursts nos primeiros {max_steps} passos...")
    burst_detected = False

    # Limita a verificação a 2^t passos ou ao tamanho da keystream, o que for menor
    steps_to_check = min(max_steps, len(keystream_bits) - t + 1)

    for i in range(steps_to_check):
        slice_bits = BitVecVal(int(keystream_bits[i:i+t], 2), t)

        solver_burst.push()
        solver_burst.add(slice_bits == burst_pattern)

        if solver_burst.check() == sat:
            print(f"Burst de {t} zeros encontrado na posição {i} da keystream.")
            burst_detected = True
        solver_burst.pop()

    if not burst_detected:
        print(f"Nenhum burst detectado nos primeiros {steps_to_check} passos da keystream.")

def verificar_repeticao_burst_keystream(model, keystream_length, t):
    """Verifica a repetição de um burst de `t` zeros na keystream em até `2^(t/2)` passos ou menos."""
    repeat_interval = 2 ** (t // 2)
    burst_pattern = BitVecVal(0, t)  # Padrão de burst de `t` zeros
    repeticao_detectada = False

    print("\n=== Verificação de Repetição de Burst na Keystream ===")

    for step in range(keystream_length):
        for offset in range(1, repeat_interval + 1):
            next_step = step + offset
            if next_step >= keystream_length:
                break  # Limite de passos atingido

            # Verifica se ambos os passos têm um burst de `t` zeros
            keystream_slice_1 = 0
            keystream_slice_2 = 0
            
            for pos in range(t):
                if step + pos < keystream_length:
                    next_bit_1 = model[BitVec(f'keystream_{step + pos}', 1)]
                    if next_bit_1 == 0:
                        keystream_slice_1 += 1  # Incrementa a contagem se for zero

                if next_step + pos < keystream_length:
                    next_bit_2 = model[BitVec(f'keystream_{next_step + pos}', 1)]
                    if next_bit_2 == 0:
                        keystream_slice_2 += 1  # Incrementa a contagem se for zero

            if keystream_slice_1 == t and keystream_slice_2 == t:
                print(f"Repetição detectada na keystream entre os passos {step} e {next_step}.")
                repeticao_detectada = True
                break  # Sai do loop quando uma repetição é encontrada

    if not repeticao_detectada:
        print("Nenhuma repetição de burst detectada na keystream.")

# Atualizando a função gera_traço para chamar as novas funções de verificação
def gera_traço(keystream_length=228):
    """Executa a simulação completa do gerador da chave A5/1"""
    print("A iniciar simulação do gerador A5/1...")

    solver, R1, R2, R3 = init()
    current_step = 0
    session_key = BitVecVal(0b0100111000101111010011010111110000011110101110001000101100111010, 64)
    frame_counter = BitVecVal(0b1110101011001111001011, 22)
    
    # Combinações com a session_key (64 iterações)
    print("\nAdicionando restrições de combinações da session_key...")
    for i in range(64):
        key_bit = Extract(63 - i, 63 - i, session_key)
        R1, R2, R3 = trans(solver, current_step, R1, R2, R3, key_bit=key_bit)
        current_step += 1
    
    # Combinações com o frame counter (22 iterações)
    print("\nAdicionando restrições de combinações do frame counter...")
    for i in range(22):
        fc_bit = Extract(21 - i, 21 - i, frame_counter)
        R1, R2, R3 = trans(solver, current_step, R1, R2, R3, fc_bit=fc_bit)
        current_step += 1
    
    # Clocking pela regra da maioria (100 iterações)
    print("\nAdicionando restrições de clocking com regra da maioria...")
    for _ in range(100):
        R1, R2, R3 = trans(solver, current_step, R1, R2, R3, majority_rule=True)
        current_step += 1

    # Geração da keystream
    print("\nAdicionando restrições para a geração da keystream...")
    for _ in range(keystream_length):
        R1, R2, R3 = trans(solver, current_step, R1, R2, R3, generate_keystream=True)
        current_step += 1
    
    print("\nA verificar satisfiabilidade...")
    if solver.check() == sat:
        print("Caminho válido encontrado! A imprimir estados...")
        model = solver.model()

        # Sessão de mistura com chave de sessão
        print("\n=== Mistura com Chave de Sessão ===")
        for i in range(65):  # De 0 a 64
            print_state_with_keystream(model, i, 19, 22, 23)

        # Sessão de mistura com frame counter
        print("\n=== Mistura com Frame Counter ===")
        for i in range(65, 87):  # De 65 a 86 (22 iterações)
            print_state_with_keystream(model, i, 19, 22, 23)

        # Clocking pela regra da maioria
        print("\n=== Clocking pela Regra da Maioria ===")
        for i in range(87, 187):  # De 87 até 186 (100 iterações)
            print_state_with_keystream(model, i, 19, 22, 23)

        # Geração da keystream com visualização passo a passo
        print("\n=== Geração da Keystream ===")
        print("A iniciar geração de", keystream_length, "bits da keystream...")
        keystream = ""
        for i in range(187, current_step + 1):
            keystream = print_state_with_keystream(model, i, 19, 22, 23, keystream, print_keystream=True)

        
        verificar_burst_keystream(model, t=3, keystream_bits=keystream)
        verificar_repeticao_burst_keystream(model, keystream_length, t=6)  
    else:
        print("Nenhum caminho válido encontrado.")
        print("", solver.unsat_core())

gera_traço()

A iniciar simulação do gerador A5/1...

Adicionando restrições de combinações da session_key...

Adicionando restrições de combinações do frame counter...

Adicionando restrições de clocking com regra da maioria...

Adicionando restrições para a geração da keystream...

A verificar satisfiabilidade...
Caminho válido encontrado! A imprimir estados...

=== Mistura com Chave de Sessão ===

Estado 0:
R1: 1101111000111110001
R2: 1010000110100010010110
R3: 01011111101101000100000

Estado 1:
R1: 0110111100011111000
R2: 1101000011010001001011
R3: 10101111110110100010000

Estado 2:
R1: 0011011110001111100
R2: 1110100001101000100101
R3: 01010111111011010001000

Estado 3:
R1: 0001101111000111110
R2: 1111010000110100010010
R3: 10101011111101101000100

Estado 4:
R1: 1000110111100011111
R2: 1111101000011010001001
R3: 01010101111110110100010

Estado 5:
R1: 0100011011110001111
R2: 0111110100001101000100
R3: 10101010111111011010001

Estado 6:
R1: 0010001101111000111
R2: 1011111010000110100010
R3: 01010